In [ ]:
%AddDeps org.apache.spark spark-sql-kafka-0-10_2.11 2.4.3 --transitive 
%AddDeps org.apache.spark spark-streaming-kafka-0-10_2.11 2.4.3 --transitive
%AddDeps org.apache.spark spark-avro_2.11 2.4.3 --transitive

In [ ]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val base_path = "..."

val broker = "..."

val spark = SparkSession
  .builder
  .appName("pm")
  .getOrCreate()

val input_errors = spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", broker + ":9092")
  .option("subscribe", "errors-simple")
  .load()

val raw_errors = input_errors.selectExpr("CAST(value AS STRING)")

val error_schema = new StructType()
      .add("deviceID",StringType)
      .add("timestamp",StringType)
      .add("error",StringType)

val errors = raw_errors.select(from_json(col("value").cast("string"), error_schema).as("data"))
    .select("data.*")

errors
  .writeStream
  .outputMode("append")
  .format("parquet")
  .option("checkpointLocation", base_path + "checkpoint/errors/")
  .option("path", base_path + "errors.parquet")
  .start()
  .awaitTermination()